<a href="https://colab.research.google.com/github/eugenerz/auto-limpieza-notebook/blob/main/Zocalito_Formato_de_Limpieza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import pandas as pd
import gspread
from google.colab import auth
from google.auth import default
import urllib.parse
import sqlite3
from gspread_dataframe import set_with_dataframe
import numpy as np
import re
import ast
import sys
sys.path.append('/content/drive/MyDrive/Limpieza_de_datos')
import mi_libreria as ml
ml.autenticar_drive()
gc = ml.autenticar_google()
conn, cursor = ml.conectar_sqlite_memoria()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

def obtener_nuevos_registros(df_nueva, df_limpia, columna_id='ID_registro'):
    """
    Filtra los registros nuevos que no están en la base limpia.

    Parámetros:
        df_nueva (pd.DataFrame): DataFrame con la base actualizada.
        df_limpia (pd.DataFrame): DataFrame con la base ya limpia.
        columna_id (str): Nombre de la columna de ID único.

    Retorna:
        pd.DataFrame: Subconjunto de df_nueva con solo los registros nuevos.
    """
    ids_limpios = set(df_limpia[columna_id])
    nuevos_registros = df_nueva[~df_nueva[columna_id].isin(ids_limpios)].copy()
    return nuevos_registros

## CARGADO DE BASES


In [ ]:

## BASE DE RESPUESTAS
sheet_id = "1PHXA-6UKh4XmHE6yy8zYvMiYAgw_D1u8iyyE4T6Tp3c"

sheet_name = "Respuestas"


url1 = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
base_raw=pd.read_csv(url1, encoding='UTF8', dtype=str)







In [ ]:

## BASE DE CODIGOS
sheet_id2 = "1m5DYExVVMKglqjFFMrsp1oDMXS_PkLKEJIKg_QMo6-M"

sheet_name2 = "Codigos"


url1 = f"https://docs.google.com/spreadsheets/d/{sheet_id2}/gviz/tq?tqx=out:csv&sheet={sheet_name2}"
codigos=pd.read_csv(url1, encoding='UTF8', dtype=str)



In [ ]:

## BASE DE CLASIFICACION
sheet_name3 = "Clasificacion"


url1 = f"https://docs.google.com/spreadsheets/d/{sheet_id2}/gviz/tq?tqx=out:csv&sheet={sheet_name3}"
clasificacion_abiertas=pd.read_csv(url1, encoding='UTF8', dtype=str)

## BASE LIMPIA

sheet_name3_1 = "Base Limpia"
sheet_name3_1_encoded = urllib.parse.quote(sheet_name3_1)


url1 = f"https://docs.google.com/spreadsheets/d/{sheet_id2}/gviz/tq?tqx=out:csv&sheet={sheet_name3_1_encoded}"
base_limpia=pd.read_csv(url1, encoding='UTF8', dtype=str)

In [ ]:
## Base de ID's


sheet_id4 = "1vAXk01VQsPJIn1WkSem_-NmiuyZKLJa163-JG8vtw2U"
sheet_name4 = "Hoja 1"

sheet_name4_encoded = urllib.parse.quote(sheet_name4)

url1 = f"https://docs.google.com/spreadsheets/d/{sheet_id4}/gviz/tq?tqx=out:csv&sheet={sheet_name4_encoded}"
base_id_equipo=pd.read_csv(url1, encoding='UTF8', dtype=str)

diccionario_correo_a_id = {}
for _, row in base_id_equipo.iterrows():
    diccionario_correo_a_id[row["CORREO"]] = row["ID"]  # Add primary email
    if pd.notna(row["CORREO ALTERNATIVO"]) and row["CORREO ALTERNATIVO"] != '':  # Add alternative email if not empty
        diccionario_correo_a_id[row["CORREO ALTERNATIVO"]] = row["ID"]

In [ ]:
diccionario_correo_a_id['cervantesfloresj41@gmail.com'] = 'E7'

In [ ]:
diccionario_correo_a_id

{'alvaradoignacio850@gmail.com': 'E1',
 'luisalbertobarcenas288@gmail.com': 'E2',
 'jorgembg74@gmail.com': 'C1',
 'casamisabella@gmail.com': 'E3',
 'rosariocarmonatapia6@gmail.com': 'E4',
 'robert0213@hotmail.com': 'C2',
 'crepusculoluna73@gmail.com': 'E5',
 'castillocasgab@gmail.com': 'E6',
 'parcerosk08@gmail.com': 'C3',
 'josefinacerzf@gmail.com': 'E7',
 'josefinacervantes1457@gmail.com': 'E7',
 'laboralbusiness@gmail.com': 'E8',
 'laboral.bissniness@gmail.com': 'E8',
 'armandocontrerasgonzalez1@gmail.com': 'E9',
 'davidcorona808@gmail.com': 'E10',
 'dmtrabajo4@gmail.com': 'E10',
 'madomingug1994@gmail.com': 'E11',
 'fespindola752@gmail.com': 'E12',
 'fannyespindola7@gmail.com': 'E12',
 'fagacdmx@gmail.com': 'E13',
 'afrancoglez19@gmail.com': 'E13',
 'garridosusylu@gmail.com': 'E14',
 'cgpigabriela.83@gmail.com': 'E14',
 'r33613702@gmail.com': 'E15',
 'victor.m.guadarrama.garcia@gmail.com': 'C4',
 'blancageredmujeres@gmail.com': 'E16',
 'janetjulieta79@gmail.com': 'E17',
 'janethcdm

## FORMATEO DE TITUTLOS Y IDS

In [ ]:
base=base_raw.copy()

In [ ]:
columnas=ml.extraer_headers(base)
base.columns=columnas["Codigo"]
base['index']=base.index
base.insert(0,'index',base.pop('index'))

In [ ]:
base['Marca temporal'] = pd.to_datetime(base['Marca temporal'], errors='coerce')
base['Fecha'] = base['Marca temporal'].dt.date
base['Hora'] = base['Marca temporal'].dt.time

<ipython-input-1063-d7552457d62b>:1: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  base['Marca temporal'] = pd.to_datetime(base['Marca temporal'], errors='coerce')


In [ ]:
base.insert(1,'Fecha',base.pop('Fecha'))
base.insert(2,'Hora',base.pop('Hora'))
base['Hora'] = base['Hora'].astype(str).str[:5]

In [ ]:
base["Marca temporal"] = pd.to_datetime(base["Marca temporal"], errors="coerce")
base["ID_registro"] = base["Marca temporal"].dt.strftime('%Y%m%d%H%M%S')
base.insert(1, 'ID_registro', base.pop('ID_registro'))

In [ ]:

base['ID_registro'] = base['ID_registro'] + '_' + base.index.astype(str)


In [ ]:
ml.validar_ids_unicos(base, "ID_registro")

✅ Todos los valores en la columna 'ID_registro' son únicos.


In [ ]:
base

Codigo,index,ID_registro,Fecha,Hora,Marca temporal,Dirección de correo electrónico,H,COD,S1,D1,...,D4,P19,NSE 01,NSE 02,NSE 03,NSE 04,NSE 05,NSE 06,PROTESTA,¿Por qué no se realizó encuesta?
0,0,20250430102850_0,2025-04-30,10:28,2025-04-30 10:28:50,castillocasgab@gmail.com,10:30:00 a.m.,C5. JULIO CESAR MARTINEZ MEDINA,SI,1. Masculino,...,1. Ama de casa (hogar),3. Media,5. Secundaria completa,2. Uno,1. Ninguno,1. Si tiene,3. Dos,2. Una,ACEPTO,NaN
1,1,20250430102901_1,2025-04-30,10:29,2025-04-30 10:29:01,laboral.bissniness@gmail.com,10:25:00 a.m.,C4. VICTOR MANUEL GUADARRAMA GARCIA,SI,2. Femenino,...,1. Ama de casa (hogar),3. Media,5. Secundaria completa,2. Uno,1. Ninguno,1. Si tiene,2. Una,2. Una,ACEPTO,NaN
2,2,20250430103315_2,2025-04-30,10:33,2025-04-30 10:33:15,verojuancdmx@gmail.com,10:24:00 a.m.,C5. JULIO CESAR MARTINEZ MEDINA,SI,1. Masculino,...,14. Empleado empresa privada,3. Media,3. Primaria completa,3. Dos o más,1. Ninguno,1. Si tiene,2. Una,2. Una,ACEPTO,NaN
3,3,20250430103329_3,2025-04-30,10:33,2025-04-30 10:33:29,fannyespindola7@gmail.com,10:28:00 a.m.,C2. ROBERTO CARLOS CASASOLA TECUA,SI,2. Femenino,...,1. Ama de casa (hogar),3. Media,9. Preparatoria completa,3. Dos o más,1. Ninguno,1. Si tiene,5. Cuatro o más,4. Tres,ACEPTO,NaN
4,4,20250430103334_4,2025-04-30,10:33,2025-04-30 10:33:34,laboral.bissniness@gmail.com,10:29:00 a.m.,C4. VICTOR MANUEL GUADARRAMA GARCIA,SI,2. Femenino,...,1. Ama de casa (hogar),3. Media,9. Preparatoria completa,2. Uno,1. Ninguno,1. Si tiene,2. Una,3. Dos,ACEPTO,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1058,1058,20250430200105_1058,2025-04-30,20:01,2025-04-30 20:01:05,minervapavon17@gmail.com,8:00:00 p.m.,C1. JORGE MARTIN BERNACHI GARCIA,No,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3. Rechazó la encuesta
1059,1059,20250430200111_1059,2025-04-30,20:01,2025-04-30 20:01:11,lilianmendozarodriguezcdmx2022@gmail.com,7:55:00 p.m.,C9. MARCO ANTONIO VILLALOBOS HUERTA,SI,2. Femenino,...,1. Ama de casa (hogar),4. Media baja,5. Secundaria completa,2. Uno,1. Ninguno,1. Si tiene,2. Una,3. Dos,ACEPTO,NaN
1060,1060,20250430200139_1060,2025-04-30,20:01,2025-04-30 20:01:39,armandocontrerasgonzalez1@gmail.com,7:58:00 p.m.,C8. ALCIBIADES POLANCO ORTIZ,No,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3. Rechazó la encuesta
1061,1061,20250430200156_1061,2025-04-30,20:01,2025-04-30 20:01:56,armandocontrerasgonzalez1@gmail.com,8:00:00 p.m.,C8. ALCIBIADES POLANCO ORTIZ,No,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3. Rechazó la encuesta


## Limpieza

In [ ]:
base['Dirección de correo electrónico']=base['Dirección de correo electrónico'].map(diccionario_correo_a_id).fillna(base['Dirección de correo electrónico'])
base.rename(columns={'Dirección de correo electrónico': 'ID Encuestador'}, inplace=True)

In [ ]:
codigos['Tipo de Pregunta'].value_counts()

,count
Tipo de Pregunta,
MULTIPLE,48
ABIERTA,10
INDICADORES,4
ABIERTA (NO CLASIFICABLE),1
DE REFERENCIA,1


In [ ]:

# Filtra las filas donde 'Tipo de Pregunta' es 'MULTIPLE' o 'DE REFERENCIA'
multiple_reference_rows = codigos[codigos['Tipo de Pregunta'].isin(['MULTIPLE', 'DE REFERENCIA'])]

# Obtiene los códigos de esas filas
columnas_a_procesar = multiple_reference_rows['Codigo'].tolist()



In [ ]:
## extraccion de codigos de repsuesta en multiples
for col in columnas_a_procesar:
    base[col] = base[col].apply(lambda x: ml.extraer_antes_del_punto(x))


In [ ]:
base.replace("nan", "", inplace=True)
base.replace("Nan", "", inplace=True)
base.replace("NAN", "", inplace=True)
base.replace(np.nan, "", inplace=True)

In [ ]:
base_updated=obtener_nuevos_registros(base, base_limpia, columna_id='ID_registro')

In [ ]:
base

Codigo,index,ID_registro,Fecha,Hora,Marca temporal,ID Encuestador,H,COD,S1,D1,...,D4,P19,NSE 01,NSE 02,NSE 03,NSE 04,NSE 05,NSE 06,PROTESTA,¿Por qué no se realizó encuesta?
0,0,20250430102850_0,2025-04-30,10:28,2025-04-30 10:28:50,E6,10:30:00 a.m.,C5,SI,1,...,1,3,5,2,1,1,3,2,ACEPTO,
1,1,20250430102901_1,2025-04-30,10:29,2025-04-30 10:29:01,E8,10:25:00 a.m.,C4,SI,2,...,1,3,5,2,1,1,2,2,ACEPTO,
2,2,20250430103315_2,2025-04-30,10:33,2025-04-30 10:33:15,E23,10:24:00 a.m.,C5,SI,1,...,14,3,3,3,1,1,2,2,ACEPTO,
3,3,20250430103329_3,2025-04-30,10:33,2025-04-30 10:33:29,E12,10:28:00 a.m.,C2,SI,2,...,1,3,9,3,1,1,5,4,ACEPTO,
4,4,20250430103334_4,2025-04-30,10:33,2025-04-30 10:33:34,E8,10:29:00 a.m.,C4,SI,2,...,1,3,9,2,1,1,2,3,ACEPTO,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1058,1058,20250430200105_1058,2025-04-30,20:01,2025-04-30 20:01:05,E28,8:00:00 p.m.,C1,No,,...,,,,,,,,,,3
1059,1059,20250430200111_1059,2025-04-30,20:01,2025-04-30 20:01:11,E26,7:55:00 p.m.,C9,SI,2,...,1,4,5,2,1,1,2,3,ACEPTO,
1060,1060,20250430200139_1060,2025-04-30,20:01,2025-04-30 20:01:39,E9,7:58:00 p.m.,C8,No,,...,,,,,,,,,,3
1061,1061,20250430200156_1061,2025-04-30,20:01,2025-04-30 20:01:56,E9,8:00:00 p.m.,C8,No,,...,,,,,,,,,,3


In [ ]:
ml.agregar_dataframe_a_hoja(sheet_id2,base_updated,"Base Limpia",gc)

DataFrame agregado a partir de la fila 1065 en la hoja 'Base Limpia'.


## EXTRACION DE PREGUNTAS ABIERTAS

In [ ]:
try:
    del codigos_abiertas
except NameError:
    pass  # Ignore if the variable doesn't exist



In [ ]:
# Filtra las filas donde 'Tipo de Pregunta' es 'MULTIPLE' o 'DE REFERENCIA'
multiple_reference_rows = codigos[codigos['Tipo de Pregunta'].isin(['ABIERTA', 'DE REFERENCIA'])]

# Obtiene los códigos de esas filas
codigos_abiertas = multiple_reference_rows['Codigo'].tolist()

In [ ]:
codigos_abiertas.append('Fecha')
codigos_abiertas.append('Hora')
codigos_abiertas.append('index')
codigos_abiertas.append('ID_registro')

In [ ]:
base_abiertas=base[codigos_abiertas]

In [ ]:
base_abiertas

Codigo,P3_10_otro,P7_1,P7_2,P8,P9,P13,P14,P15,P16,P17,P18,Fecha,Hora,index,ID_registro
0,,Por qué sirve para distracción,Nada más,Ninguna,Juegos,Pinta carita,Ninguna,1,Para tener distracción,Ninguna,Albercas,2025-04-30,10:28,0,20250430102850_0
1,,Dibujo,Ns,Ns,Sibujo,Dinujo,Ns,1,De desapegan del cel,Ns,Ns,2025-04-30,10:29,1,20250430102901_1
2,99,DOMINO,GENGA,GENGA,99,DOMINO,GENGA,1,SE ENTRETIENEN,99,ALBERCAS,2025-04-30,10:33,2,20250430103315_2
3,99,Tatuajes,Linea de tiempo,Trampolines,Ninguna,Trampolines,99,1,Los saca de su sona de confor,Más organización,Manualidades para todo tipo de edades,2025-04-30,10:33,3,20250430103329_3
4,,Pinta caritas,Ns,Ns,Ns,Pinta caritas,Ns,1,Los motiva ha hacer diferentes actividades,Ns,El 10 de mayo,2025-04-30,10:33,4,20250430103334_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1058,,,,,,,,,,,,2025-04-30,20:01,1058,20250430200105_1058
1059,,Inflables,99,Inflables,99,Inflable,99,1,Aprenden cosas,Ninguna,Día. Delasmafres,2025-04-30,20:01,1059,20250430200111_1059
1060,,,,,,,,,,,,2025-04-30,20:01,1060,20250430200139_1060
1061,,,,,,,,,,,,2025-04-30,20:01,1061,20250430200156_1061


In [ ]:
# prompt: añadir una columna vacia a lado de cada columna en base_abiertas cuyo Tipo de pregunta en codigos sea ABIERTA, el nombre de la columna debe ser del formato columna_clas

for col in base_abiertas.columns:
    tipo_pregunta = codigos[codigos['Codigo'] == col]['Tipo de Pregunta'].iloc[0] if col in codigos['Codigo'].values else None
    if tipo_pregunta == 'ABIERTA':
        nueva_col = col + '_clas'
        base_abiertas.insert(base_abiertas.columns.get_loc(col) + 1, nueva_col, '')


In [ ]:
base_abiertas_updated=obtener_nuevos_registros(base_abiertas, clasificacion_abiertas, columna_id='ID_registro')

In [ ]:
gc = ml.autenticar_google()
ml.agregar_dataframe_a_hoja(sheet_id2,base_abiertas_updated,"Clasificacion",gc)

DataFrame agregado a partir de la fila 1065 en la hoja 'Clasificacion'.


# Generar Base final

In [ ]:

### Procesamiento de intentos
## 03 Rechazó la encuesta
## * 04 Informante inadecuado

In [ ]:
base_intentos=base.copy()

In [ ]:
base_intentos.loc[(base_intentos["PROTESTA"] == "ACEPTO") & (base_intentos["S1"] == "SI"), "Tipo de Respuesta"] = "01"
base_intentos.loc[(base_intentos["¿Por qué no se realizó encuesta?"] == "3"),"Tipo de Respuesta"] = "03"
base_intentos.loc[(base_intentos["¿Por qué no se realizó encuesta?"] == "4"),"Tipo de Respuesta"] = "04"

In [ ]:
base_intentos["Tipo de Respuesta"].value_counts()

,count
Tipo de Respuesta,
01,937
03,125
04,1


In [ ]:
### Procesamiento de intentos

In [ ]:
import sqlite3
df=base_intentos.copy()
conn = sqlite3.connect("mi_base.db")
cursor = conn.cursor()
conn = sqlite3.connect(":memory:")
df.to_sql("df", conn, if_exists="replace", index=False)

1063

In [ ]:
query = '''
WITH base AS (
    SELECT
        *,
        CASE WHEN "Tipo de Respuesta" = '01' THEN 1 ELSE 0 END AS es_exito
    FROM df
),
grupos AS (
    SELECT
        *,
        SUM(es_exito) OVER (
            PARTITION BY 'ID Encuestador'
            ORDER BY Hora
            ROWS UNBOUNDED PRECEDING
        ) AS grupo_exito
    FROM base
),
numerados AS (
    SELECT
        *,
        ROW_NUMBER() OVER (
            PARTITION BY  'ID Encuestador', grupo_exito
            ORDER BY Hora
        ) AS etiqueta_raw
    FROM grupos
)
SELECT *
FROM numerados
ORDER BY 'ID Encuestador', Hora;
'''
df = pd.read_sql_query(query, conn)

def procesar_etiquetas(df):
    df = df.sort_values(by=["ID Encuestador", "Hora"]).reset_index(drop=True)

    def etiquetar_grupo(grupo):
        etiquetas = []
        contador = 1
        estado = "contando"

        for _, row in grupo.iterrows():
            tipo = row["Tipo de Respuesta"]

            if estado == "contando":
                etiquetas.append(contador)
                if tipo == "01":
                    contador = 1
                else:
                    contador += 1
                    if contador > 9:
                        estado = "esperando_cierre"

            elif estado == "esperando_cierre":
                if tipo == "01":
                    etiquetas.append(10)
                    estado = "contando"
                    contador = 1
                else:
                    etiquetas.append(None)

        return etiquetas

    df["etiqueta"] = (
        df.groupby(["ID Encuestador"], group_keys=False)
          .apply(etiquetar_grupo)
          .explode()
          .reset_index(drop=True)
    )

    return df





df = procesar_etiquetas(df)
df[['Fecha','Hora','ID Encuestador','Tipo de Respuesta','etiqueta']]



<ipython-input-1091-9c51f561d54e>:65: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(etiquetar_grupo)


,Fecha,Hora,ID Encuestador,Tipo de Respuesta,etiqueta
0,2025-04-30,15:40,E10,03,1
1,2025-04-30,15:47,E10,01,2
2,2025-04-30,15:53,E10,01,1
3,2025-04-30,16:02,E10,01,1
4,2025-04-30,16:09,E10,01,1
...,...,...,...,...,...
1058,2025-04-30,19:56,E9,03,None
1059,2025-04-30,19:59,E9,03,None
1060,2025-04-30,20:01,E9,03,None
1061,2025-04-30,20:01,E9,03,None


In [ ]:
df['etiqueta'].value_counts()

,count
etiqueta,
1,941
2,74
3,25
4,8
5,4
6,2
7,2
8,1
9,1


In [ ]:
def construir_intentos_con_todos_los_exitos(df):

    df_intentos = df[df["etiqueta"].notnull()].copy()
    df_intentos["etiqueta"] = df_intentos["etiqueta"].astype(int)


    df_intentos["grupo_local"] = (
        (df_intentos["etiqueta"] == 1) &
        (df_intentos["Tipo de Respuesta"].notna())
    ).astype(int)

    df_intentos["grupo_local"] = df_intentos.groupby(
        ["ID Encuestador"]
    )["grupo_local"].cumsum()


    df_intentos["grupo_id"] = (
        df_intentos["ID Encuestador"].astype(str) + "_" +
        #df_intentos["Fecha"].astype(str) + "_" +
        df_intentos["grupo_local"].astype(str)
    )

    # Crear columnas int01 a int10 a partir de etiqueta
    df_pivot = df_intentos.pivot_table(
        index="grupo_id",
        columns="etiqueta",
        values="Tipo de Respuesta",
        aggfunc="first"
    )


    for i in range(1, 11):
        if i not in df_pivot.columns:
            df_pivot[i] = pd.NA

    df_pivot = df_pivot[[i for i in range(1, 11)]]
    df_pivot.columns = [f"int{str(i).zfill(2)}" for i in range(1, 11)]

    # Grupo completo = si al menos hay int02 (más de 1 intento)
    df_pivot["grupo_completo"] = df_pivot["int02"].notna()

    df_pivot = df_pivot.reset_index()


    # Casos exitosos (Tipo de Respuesta = '01') con todas sus columnas
    casos_exito = df_intentos[df_intentos["Tipo de Respuesta"] == "01"].copy()
    casos_exito["grupo_id"] = casos_exito["grupo_id"].astype(str)
    df_pivot["grupo_id"] = df_pivot["grupo_id"].astype(str)


    # Merge con las columnas originales + intentos
    resultado = casos_exito.merge(
        df_pivot,
        on="grupo_id",
        how="left",
        suffixes=("", "_intentos")
    )





    return resultado


In [ ]:
df_resultado = construir_intentos_con_todos_los_exitos(df)

In [ ]:
df_resultado.columns

Index(['index', 'ID_registro', 'Fecha', 'Hora', 'Marca temporal',
       'ID Encuestador', 'H', 'COD', 'S1', 'D1', 'D2', 'D5', 'D6', 'P1',
       'P2_1', 'P2_2', 'P2_3', 'P2_5', 'P3_1', 'P3_2', 'P3_3', 'P3_4', 'P3_5',
       'P3_6', 'P3_7', 'P3_8', 'P3_9', 'P3_10', 'P3_10_otro', 'P4', 'P5', 'P6',
       'P7_1', 'P7_2', 'P8', 'P9', 'P10_1', 'P10_2', 'P10_3', 'P10_4', 'P10_5',
       'P10_6', 'P10_7', 'P11_1', 'P11_2', 'P11_3', 'P11_4', 'P11_5', 'P11_6',
       'P11_7', 'P12', 'P13', 'P14', 'P15', 'P16', 'P17', 'P18', 'D3', 'D4',
       'P19', 'NSE 01', 'NSE 02', 'NSE 03', 'NSE 04', 'NSE 05', 'NSE 06',
       'PROTESTA', '¿Por qué no se realizó encuesta?', 'Tipo de Respuesta',
       'es_exito', 'grupo_exito', 'etiqueta_raw', 'etiqueta', 'grupo_local',
       'grupo_id', 'int01', 'int02', 'int03', 'int04', 'int05', 'int06',
       'int07', 'int08', 'int09', 'int10', 'grupo_completo'],
      dtype='object')

In [ ]:
df_resultado.drop(columns=['es_exito'], inplace=True)
df_resultado.drop(columns=['grupo_exito'], inplace=True)
df_resultado.drop(columns=['etiqueta_raw'], inplace=True)
df_resultado.drop(columns=['etiqueta'], inplace=True)
df_resultado.drop(columns=['grupo_local'], inplace=True)
df_resultado.drop(columns=['grupo_id'], inplace=True)
df_resultado.drop(columns=['PROTESTA'], inplace=True)
df_resultado.drop(columns=['¿Por qué no se realizó encuesta?'], inplace=True)
df_resultado.drop(columns=['S1'], inplace=True)
df_resultado.drop(columns=['Tipo de Respuesta'], inplace=True)
df_resultado.drop(columns=['grupo_completo'], inplace=True)


In [ ]:
df_resultado.to_csv('df_resultado.csv', index=False)

In [ ]:
### Union de clasificacion

In [ ]:
clasificacion_abiertas.replace(np.nan, "", inplace=True)
#clasificacion_abiertas

In [ ]:
conn = sqlite3.connect("mi_base.db")
cursor = conn.cursor()


In [ ]:
df_resultado.to_sql("base", conn, if_exists="replace", index=False)
clasificacion_abiertas.to_sql("base_abiertas", conn, if_exists="replace", index=False)

1063

In [ ]:
query = '''

SELECT
    c1.*,
    c2.'P3_10_clas_clas',
    c2.'P7_1_clas',
    c2.'P7_2_clas',
    c2.'P8_clas',
    c2.'P9_clas',
    c2.'P13_clas',
    c2.'P14_clas',
    c2.'P16_clas',
    c2.'P17_clas',
    c2.'P18_clas'

FROM base as c1
LEFT JOIN base_abiertas as c2
ON c1."ID_registro" = c2.'ID_registro'
'''
base2 = pd.read_sql_query(query, conn)

In [ ]:
ml.validar_ids_unicos(base2, "ID_registro")

✅ Todos los valores en la columna 'ID_registro' son únicos.


In [ ]:
base2['P3_10_otro'] = base2['P3_10_clas_clas']
base2['P7_1'] = base2['P7_1_clas']
base2['P7_2'] = base2['P7_2_clas']
base2['P8'] = base2['P8_clas']
base2['P9'] = base2['P9_clas']
base2['P13'] = base2['P13_clas']
base2['P14'] = base2['P14_clas']
base2['P16'] = base2['P16_clas']
base2['P17'] = base2['P17_clas']
base2['P18'] = base2['P18_clas']

base2.drop(columns=['P3_10_clas_clas'], inplace=True)
base2.drop(columns=['P7_1_clas'], inplace=True)
base2.drop(columns=['P7_2_clas'], inplace=True)
base2.drop(columns=['P8_clas'], inplace=True)
base2.drop(columns=['P9_clas'], inplace=True)
base2.drop(columns=['P13_clas'], inplace=True)
base2.drop(columns=['P14_clas'], inplace=True)
base2.drop(columns=['P16_clas'], inplace=True)
base2.drop(columns=['P17_clas'], inplace=True)
base2.drop(columns=['P18_clas'], inplace=True)



In [ ]:
base2.rename(columns={'P3_10_clas_clas': 'P3_10_otro'}, inplace=True)
base2.rename(columns={'P7_1_clas': 'P7_1'}, inplace=True)
base2.rename(columns={'P7_2_clas': 'P7_2'}, inplace=True)
base2.rename(columns={'P8_clas': 'P8'}, inplace=True)
base2.rename(columns={'P9_clas': 'P9'}, inplace=True)
base2.rename(columns={'P13_clas': 'P13'}, inplace=True)
base2.rename(columns={'P14_clas': 'P14'}, inplace=True)
base2.rename(columns={'P16_clas': 'P16'}, inplace=True)
base2.rename(columns={'P17_clas': 'P17'}, inplace=True)
base2.rename(columns={'P18_clas': 'P18'}, inplace=True)


In [ ]:
multiple_reference_rows = codigos[codigos['Tipo de Pregunta'].isin(['ABIERTA'])]

# Obtiene los códigos de esas filas
columnas_a_procesar = multiple_reference_rows['Codigo'].tolist()
columnas_a_procesar


['P3_10_otro', 'P7_1', 'P7_2', 'P8', 'P9', 'P13', 'P14', 'P16', 'P17', 'P18']

In [ ]:
## extraccion de codigos de repsuesta en multiples
for col in columnas_a_procesar:
    base2[col] = base2[col].apply(lambda x: ml.extraer_antes_del_punto(x))

In [ ]:
bloque=['P7_1','P7_2']

In [ ]:
base2=ml.reordenar_menciones(base2,bloque,ns="96")
base2=ml.reordenar_menciones(base2,bloque,ns="97")
base2=ml.reordenar_menciones(base2,bloque,ns="98")
base2=ml.reordenar_menciones(base2,bloque,ns="99")

In [ ]:
# prompt: en base2['P15'] cambiar los valores 3 por 9

# Assuming 'base2' DataFrame is already loaded as in your provided code.

base2['P15'] = base2['P15'].replace('3', '9')
base2['P10_1'] = base2['P10_1'].replace('3', '9')
base2['P10_2'] = base2['P10_2'].replace('3', '9')
base2['P10_3'] = base2['P10_3'].replace('3', '9')
base2['P10_4'] = base2['P10_4'].replace('3', '9')
base2['P10_5'] = base2['P10_5'].replace('3', '9')
base2['P10_6'] = base2['P10_6'].replace('3', '9')
base2['P10_7'] = base2['P10_7'].replace('3', '9')


In [ ]:
base2.columns

Index(['index', 'ID_registro', 'Fecha', 'Hora', 'Marca temporal',
       'ID Encuestador', 'H', 'COD', 'D1', 'D2', 'D5', 'D6', 'P1', 'P2_1',
       'P2_2', 'P2_3', 'P2_5', 'P3_1', 'P3_2', 'P3_3', 'P3_4', 'P3_5', 'P3_6',
       'P3_7', 'P3_8', 'P3_9', 'P3_10', 'P3_10_otro', 'P4', 'P5', 'P6', 'P7_1',
       'P7_2', 'P8', 'P9', 'P10_1', 'P10_2', 'P10_3', 'P10_4', 'P10_5',
       'P10_6', 'P10_7', 'P11_1', 'P11_2', 'P11_3', 'P11_4', 'P11_5', 'P11_6',
       'P11_7', 'P12', 'P13', 'P14', 'P15', 'P16', 'P17', 'P18', 'D3', 'D4',
       'P19', 'NSE 01', 'NSE 02', 'NSE 03', 'NSE 04', 'NSE 05', 'NSE 06',
       'int01', 'int02', 'int03', 'int04', 'int05', 'int06', 'int07', 'int08',
       'int09', 'int10'],
      dtype='object')

In [ ]:
saltos_columns=codigos[['Codigo']].loc[codigos['Salto']=='SI']


In [ ]:
for column in saltos_columns['Codigo']:
    saltos_value=ml.replace_values(base2, column, ast.literal_eval(codigos.loc[codigos['Codigo'] == column, 'Saltos'].iloc[0]))


In [ ]:
#base2.drop(columns=['Marca temporal'], inplace=True)
#base2.drop(columns=['index'], inplace=True)
#base2.drop(columns=['ID_registro'], inplace=True)
base2.drop(columns=['H'], inplace=True)

In [ ]:
base2.replace("nan", "", inplace=True)
base2.replace(np.nan, "", inplace=True)
base2.replace("None", "", inplace=True)

In [ ]:

base2.loc[base2['P3_10_otro'] == '8', 'P3_9'] = '1'
base2.loc[base2['P3_10_otro'] == '8', 'P3_10_otro'] = ''

base2.loc[base2['P3_10_otro'] == '10', 'P3_1'] = '1'
base2.loc[base2['P3_10_otro'] == '10', 'P3_10_otro'] = ''

base2.loc[base2['P3_10_otro'] == '11', 'P3_2'] = '1'
base2.loc[base2['P3_10_otro'] == '11', 'P3_10_otro'] = ''

base2.loc[base2['P3_10_otro'] == '12', 'P3_7'] = '1'
base2.loc[base2['P3_10_otro'] == '12', 'P3_10_otro'] = ''

base2.loc[base2['P3_10_otro'] == '13', 'P3_3'] = '1'
base2.loc[base2['P3_10_otro'] == '13', 'P3_10_otro'] = ''

base2.loc[(base2['P3_10'] == '2') & (base2['P3_10_otro']!=""), 'P3_10'] = '1'
base2.loc[(base2['P3_10'] == '1') & (base2['P3_10_otro']==""), 'P3_10'] = '2'
base2.loc[(base2['P3_10'] == '1') & (base2['P3_10_otro']=="99"), 'P3_10'] = '2'
base2.loc[(base2['P3_10'] == '2') & (base2['P3_10_otro']=="99"), 'P3_10_otro'] = ''







In [ ]:

# Sort the DataFrame by the 'index' column in ascending order
base2 = base2.sort_values(by='index', ascending=True)

# Reset the index
base2 = base2.reset_index(drop=True)

# Now 'base2' is sorted by 'index' in ascending order


In [ ]:
base2.drop(columns=['index'], inplace=True)

base2.drop(columns=['Marca temporal'], inplace=True)


In [ ]:


# Reorder columns int01 to int10 from column 03 onwards, maintaining ascending order
int_columns = [col for col in base2.columns if re.match(r'int\d{2}', col)]
other_columns = [col for col in base2.columns if not re.match(r'int\d{2}', col)]

# Find the index of the third column
third_column_index = 5

# Create the new column order
new_columns = other_columns[:third_column_index] + int_columns + other_columns[third_column_index:]

# Reorder the DataFrame columns
base2 = base2[new_columns]


In [ ]:
base2.rename(columns={'COD': 'ID_Coordinador'}, inplace=True)
base2.rename(columns={'ID Encuestador': 'ID_Encuestador'}, inplace=True)
base2.rename(columns={'NSE 01': 'NSE_01'}, inplace=True)
base2.rename(columns={'NSE 02': 'NSE_02'}, inplace=True)
base2.rename(columns={'NSE 03': 'NSE_03'}, inplace=True)
base2.rename(columns={'NSE 04': 'NSE_04'}, inplace=True)
base2.rename(columns={'NSE 05': 'NSE_05'}, inplace=True)
base2.rename(columns={'NSE 06': 'NSE_06'}, inplace=True)



In [ ]:


base2.loc[base2['P3_10_otro'] == '9', 'P3_10_otro'] = '8'


In [ ]:
ml.validar_ids_unicos(base2, "ID_registro")

✅ Todos los valores en la columna 'ID_registro' son únicos.


In [ ]:
base2.drop(columns=['ID_registro'], inplace=True)

In [ ]:
base2.to_csv('Zocalito-Base Final.csv',index=False)

In [ ]:
#ml.agregar_dataframe_a_hoja(sheet_id2,base2,"Base Zocalito Final",gc)

In [ ]:


# Assuming 'base2' DataFrame is already loaded and processed as in your provided code.

# Convert the DataFrame to a CSV file
base2.to_csv('Zocalito_spss.csv', index=False, encoding='utf-8-sig')

# Install the pyreadstat library for SPSS file conversion (if not already installed)
!pip install pyreadstat

# Import the necessary library
import pyreadstat

# Convert the CSV file to an SPSS (.sav) file
pyreadstat.write_sav(base2, "Zocalito_spss.sav")

# Download the .sav file from Colab
from google.colab import files
files.download("Zocalito_spss.sav")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>